In [14]:
import pandas as pd
import numpy as np
def create_classification_data(algo, dim, budgets, targets):
    for fixed_target in targets:
        for budget in budgets:
            path = 'embeddings/'+algo
            path_clas = 'classification_data/'+algo
            path_KG = 'KG/'+algo
            df_embeddings_problems = pd.read_csv(f"../Results/{path}/embeddings_problems_ComplEx_dim_{dim}_budget_{budget}_target_{fixed_target:.5f}.csv")
            df_embeddings_confs = pd.read_csv(f"../Results/{path}/embeddings_confs_ComplEx_dim_{dim}_budget_{budget}_target_{fixed_target:.5f}.csv")
            df_embeddings_merged = df_embeddings_confs.merge(df_embeddings_problems, how='cross')
            df_embeddings_merged['conf_f_i'] = df_embeddings_merged.apply(lambda row: row.conf+"_"+row.problem, axis=1)
            df_embeddings_merged = df_embeddings_merged.set_index('conf_f_i')    
            df_embeddings_merged.drop(['conf', 'problem'], axis=1, inplace=True)
            
            KG_train_links = np.load(f"../DATA/{path_KG}/KG_dim_{dim}_budget_{budget}_target_{fixed_target:.5f}_train_links.npy")
            KG_val_links = np.load(f"../DATA/{path_KG}/KG_dim_{dim}_budget_{budget}_target_{fixed_target:.5f}_val.npy")
            KG_train_links = np.concatenate((KG_train_links, KG_val_links))
            sub_train = [triplet[0]+"_"+triplet[2] for triplet in KG_train_links]
            solved_train = [triplet[1] for triplet in KG_train_links]
            df_embeddings_merged_train = df_embeddings_merged.loc[sub_train]
            df_embeddings_merged_train['solved'] = solved_train
            df_embeddings_merged_train.to_csv(f'../Results/{path_clas}/ComplEx_dim_{dim}_budget_{budget}_target_{fixed_target:.5f}_train.csv')

            KG_test_links = np.load(f"../DATA/{path_KG}/KG_dim_{dim}_budget_{budget}_target_{fixed_target:.5f}_test.npy")
            sub_test = [triplet[0]+"_"+triplet[2] for triplet in KG_test_links]
            solved_test = [triplet[1] for triplet in KG_test_links]
            df_embeddings_merged_test = df_embeddings_merged.loc[sub_test]
            df_embeddings_merged_test['solved'] = solved_test
            df_embeddings_merged_test.to_csv(f'../Results/{path_clas}/ComplEx_dim_{dim}_budget_{budget}_target_{fixed_target:.5f}_test.csv')

In [18]:
create_classification_data('modDE', 30, [50000], [0.1])